In [3]:
import pandas as pd
import numpy as np
import datetime as dt
from dateutil.relativedelta import relativedelta


# from bcb import sgs

In [4]:
import json
from io import StringIO
from typing import Dict, Generator, List, Optional, Tuple, TypeAlias, Union

import pandas as pd
import requests

from bcb.utils import Date, DateInput

"""
Sistema Gerenciador de Séries Temporais (SGS)

O módulo ``sgs`` obtem os dados do webservice do Banco Central,
interface json do serviço BCData/SGS -
`Sistema Gerenciador de Séries Temporais (SGS)
<https://www3.bcb.gov.br/sgspub/localizarseries/localizarSeries.do?method=prepararTelaLocalizarSeries>`_.
"""


class SGSCode:
    def __init__(self, code: Union[str, int], name: Optional[str] = None) -> None:
        if name is None:
            if isinstance(code, int) or isinstance(code, str):
                self.name = str(code)
                self.value = int(code)
        else:
            self.name = str(name)
            self.value = int(code)

    def __repr__(self):
        return f"{self.code} - {self.name}" if self.name else f"{self.code}"


SGSCodeInput: TypeAlias = Union[
    int,
    str,
    Tuple[str, Union[int, str]],
    List[Union[int, str, Tuple[str, Union[int, str]]]],
    Dict[str, Union[int, str]],
]


def _codes(codes: SGSCodeInput) -> Generator[SGSCode, None, None]:
    if isinstance(codes, int) or isinstance(codes, str):
        yield SGSCode(codes)
    elif isinstance(codes, tuple):
        yield SGSCode(codes[1], codes[0])
    elif isinstance(codes, list):
        for cd in codes:
            _ist = isinstance(cd, tuple)
            yield SGSCode(cd[1], cd[0]) if _ist else SGSCode(cd)
    elif isinstance(codes, dict):
        for name, code in codes.items():
            yield SGSCode(code, name)


def _get_url_and_payload(code: int, start_date: DateInput, end_date: DateInput, last: int) -> Dict[str, str]:
    payload = {"formato": "json"}
    if last == 0:
        if start_date is not None or end_date is not None:
            payload["dataInicial"] = Date(start_date).date.strftime("%d/%m/%Y")
            end_date = end_date if end_date else "today"
            payload["dataFinal"] = Date(end_date).date.strftime("%d/%m/%Y")
        url = "https://api.bcb.gov.br/dados/serie/bcdata.sgs.{}/dados".format(code)
    else:
        url = ("https://api.bcb.gov.br/dados/serie/bcdata.sgs.{}/dados" "/ultimos/{}").format(code, last)

    return {"payload": payload, "url": url}


def _format_df(df: pd.DataFrame, code: SGSCode, freq: str) -> pd.DataFrame:
    cns = {"data": "Date", "valor": code.name, "datafim": "enddate"}
    df = df.rename(columns=cns)
    if "Date" in df:
        df["Date"] = pd.to_datetime(df["Date"], format="%d/%m/%Y")
    if "enddate" in df:
        df["enddate"] = pd.to_datetime(df["enddate"], format="%d/%m/%Y")
    df = df.set_index("Date")
    if freq:
        df.index = df.index.to_period(freq)
    return df



In [85]:
def get(
    codes: SGSCodeInput,
    start: Optional[DateInput] = None,
    end: Optional[DateInput] = None,
    last: int = 0,
    multi: bool = True,
    freq: Optional[str] = None,
) -> Union[pd.DataFrame, List[pd.DataFrame]]:
    """
    Retorna um DataFrame pandas com séries temporais obtidas do SGS.

    Parameters
    ----------

    codes : {int, List[int], List[str], Dict[str:int]}
        Este argumento pode ser uma das opções:

        * ``int`` : código da série temporal
        * ``list`` ou ``tuple`` : lista ou tupla com códigos
        * ``list`` ou ``tuple`` : lista ou tupla com pares ``('nome', código)``
        * ``dict`` : dicionário com pares ``{'nome': código}``

        Com códigos numéricos é interessante utilizar os nomes com os códigos
        para definir os nomes nas colunas das séries temporais.
    start : str, int, date, datetime, Timestamp
        Data de início da série.
        Interpreta diferentes tipos e formatos de datas.
    end : string, int, date, datetime, Timestamp
        Data final da série.
        Interpreta diferentes tipos e formatos de datas.
    last : int
        Retorna os últimos ``last`` elementos disponíveis da série temporal
        solicitada. Se ``last`` for maior que 0 (zero) os argumentos ``start``
        e ``end`` são ignorados.
    multi : bool
        Define se, quando mais de 1 série for solicitada, a função retorna uma
        série multivariada ou uma lista com séries univariadas.
    freq : str
        Define a frequência a ser utilizada na série temporal

    Returns
    -------

    ``DataFrame`` :
        série temporal univariada ou multivariada,
        quando solicitado mais de uma série (parâmetro ``multi=True``).

    ``list`` :
        lista com séries temporais univariadas,
        quando solicitado mais de uma série (parâmetro ``multi=False``).
    """
    dfs = []
    for code in _codes(codes):
        text = get_json(code.value, start, end, last)
        df = pd.read_json(StringIO(text))
        df = _format_df(df, code, freq)

        if check_years_interval(start, end) is True:
            df.sort_index(inplace=True)
            df = df.loc[start:]
            df = df.reset_index().drop_duplicates().set_index('Date')

        dfs.append(df)
    if len(dfs) == 1:
        return dfs[0]
    else:
        if multi:
            return pd.concat(dfs, axis=1)
        else:
            return dfs

def check_dates(start, end):
    if start is None:
        raise Exception("Date Error: Informe a data de início da(s) série(s).")
    if end is None:
        end = dt.date.today().strftime('%Y-%m-%d')
    
    return start, end

def check_years_interval(start: Optional[DateInput] = None, end: Optional[DateInput] = None, limit:int=8):
    
    start, end = check_dates(start, end)

    start_as_date = dt.datetime.strptime(start, '%Y-%m-%d').date() if type(start) == str else start
    end_as_date = dt.datetime.strptime(end, '%Y-%m-%d').date() if type(end) == str else end
    
    diff_years = (end_as_date - start_as_date).days / 365

    return diff_years > limit


def create_time_chunks(start: Optional[DateInput] = None, end: Optional[DateInput] = None, step: int = 5):
        start, end = check_dates(start, end)

        start_as_date = dt.datetime.strptime(start, '%Y-%m-%d').date() if type(start) == str else start
        end_as_date = dt.datetime.strptime(end, '%Y-%m-%d').date() if type(end) == str else end
        
        previous_start_year = (start_as_date - pd.offsets.YearBegin(1)).date()
        next_end_year = (end_as_date + pd.offsets.YearBegin(0)).date()

        date_range = pd.date_range( start=previous_start_year, 
                                    end=next_end_year, 
                                    freq=f'{step}YS', 
                                    inclusive='left').to_list()

        date_range = date_range + [end_as_date]

        return date_range

def get_json(code: int, start: Optional[DateInput] = None, end: Optional[DateInput] = None, last: int = 0) -> str:
    """
    Retorna um JSON com séries temporais obtidas do SGS.

    Parameters
    ----------

    code : int
        Código da série temporal
    start : str, int, date, datetime, Timestamp
        Data de início da série.
        Interpreta diferentes tipos e formatos de datas.
    end : string, int, date, datetime, Timestamp
        Data final da série.
        Interpreta diferentes tipos e formatos de datas.
    last : int
        Retorna os últimos ``last`` elementos disponíveis da série temporal
        solicitada. Se ``last`` for maior que 0 (zero) os argumentos ``start``
        e ``end`` são ignorados.

    Returns
    -------

    JSON :
        série temporal univariada em formato JSON.
    """
    start, end = check_dates(start, end)

    # try: 
    #     check_years_interval(start, end)

    # except:
    #     raise Exception("Date Error: Informe as datas de início e fim da(s) série(s).")

    
    # if check_years_interval(start, end) is True:  
        
    #     date_range = create_time_chunks(start, end, step=5)

    #     final_json = []
    #     for i_dates in range( len(date_range)-1 ):
    #         current_start = date_range[i_dates].strftime('%Y-%m-%d')
    #         current_end = date_range[i_dates + 1].strftime('%Y-%m-%d')
    #         print('Start:', current_start, '|', 'End:', current_end)

    #         urd = _get_url_and_payload(code, current_start, current_end, last)
    #         res = requests.get(urd["url"], params=urd["payload"])
    #         if res.status_code != 200:
    #             try:
    #                 res_json = json.loads(res.text)
    #             except Exception:
    #                 res_json = {}
    #             if "error" in res_json:
    #                 raise Exception("BCB error: {}".format(res_json["error"]))
    #             elif "erro" in res_json:
    #                 raise Exception("BCB error: {}".format(res_json["erro"]["detail"]))
    #             raise Exception("Download error: code = {}".format(code))

    #         final_json = final_json + res.text.replace('[', '').replace(']', '').split(',')
    #     final_json = '[' + ','.join(final_json) + ']'

    #     return final_json

    urd = _get_url_and_payload(code, start, end, last)
    res = requests.get(urd["url"], params=urd["payload"])
    if res.status_code != 200:
        try:
            res_json = json.loads(res.text)
        except Exception:
            res_json = {}
        if "error" in res_json:
            # raise Exception("BCB error: {}".format(res_json["error"]))
            if check_years_interval(start, end) is True:  
        
                date_range = create_time_chunks(start, end, step=5)

                final_json = []
                for i_dates in range( len(date_range)-1 ):
                    current_start = date_range[i_dates].strftime('%Y-%m-%d')
                    current_end = date_range[i_dates + 1].strftime('%Y-%m-%d')
                    print('Start:', current_start, '|', 'End:', current_end)

                    urd = _get_url_and_payload(code, current_start, current_end, last)
                    res = requests.get(urd["url"], params=urd["payload"])
                    if res.status_code != 200:
                        try:
                            res_json = json.loads(res.text)
                        except Exception:
                            res_json = {}
                    final_json = final_json + res.text.replace('[', '').replace(']', '').split(',')
            final_json = '[' + ','.join(final_json) + ']'
            return final_json

        elif "erro" in res_json:
            raise Exception("BCB error: {}".format(res_json["erro"]["detail"]))
        raise Exception("Download error: code = {}".format(code))
    return res.text
    # return start


In [147]:
# --- 1. IMPORTAÇÕES ---
import json as json_parser
from io import StringIO
from typing import Dict, Generator, List, Optional, Tuple, TypeAlias, Union
import datetime as dt

import pandas as pd
import requests
from dateutil.relativedelta import relativedelta

from bcb.utils import Date, DateInput


# --- 2. CLASSES E FUNÇÕES ORIGINAIS DA BIBLIOTECA (sem alterações) ---
class SGSCode:
    def __init__(self, code: Union[str, int], name: Optional[str] = None) -> None:
        if name is None:
            if isinstance(code, int) or isinstance(code, str):
                self.name = str(code)
                self.value = int(code)
        else:
            self.name = str(name)
            self.value = int(code)
    def __repr__(self):
        return f"{self.value} - {self.name}" if self.name else f"{self.value}"

SGSCodeInput: TypeAlias = Union[int, str, Tuple[str, Union[int, str]], List[Union[int, str, Tuple[str, Union[int, str]]]], Dict[str, Union[int, str]],]

def _codes(codes: SGSCodeInput) -> Generator[SGSCode, None, None]:
    if isinstance(codes, int) or isinstance(codes, str):
        yield SGSCode(codes)
    elif isinstance(codes, tuple):
        yield SGSCode(codes[1], codes[0])
    elif isinstance(codes, list):
        for cd in codes:
            _ist = isinstance(cd, tuple)
            yield SGSCode(cd[1], cd[0]) if _ist else SGSCode(cd)
    elif isinstance(codes, dict):
        for name, code in codes.items():
            yield SGSCode(code, name)

def _get_url_and_payload(code: int, start_date: Optional[DateInput], end_date: Optional[DateInput], last: int) -> Dict[str, str]:
    payload = {"formato": "json"}
    if last == 0:
        if start_date is not None or end_date is not None:
            payload["dataInicial"] = Date(start_date).date.strftime("%d/%m/%Y")
            end_date = end_date if end_date else "today"
            payload["dataFinal"] = Date(end_date).date.strftime("%d/%m/%Y")
        url = f"https://api.bcb.gov.br/dados/serie/bcdata.sgs.{code}/dados"
    else:
        url = f"https://api.bcb.gov.br/dados/serie/bcdata.sgs.{code}/dados/ultimos/{last}"
    return {"payload": payload, "url": url}

def _format_df(df: pd.DataFrame, code: SGSCode, freq: str) -> pd.DataFrame:
    cns = {"data": "Date", "valor": code.name, "datafim": "enddate"}
    df = df.rename(columns=cns)
    if "Date" in df:
        df["Date"] = pd.to_datetime(df["Date"], dayfirst=True)
    if "enddate" in df:
        df["enddate"] = pd.to_datetime(df["enddate"], dayfirst=True)
    df = df.set_index("Date")
    if freq:
        df.index = df.index.to_period(freq)
    return df


# --- 3. FUNÇÕES NOVAS E REATORADAS COM A LÓGICA HÍBRIDA ---

def _probe_is_daily(code: int) -> bool:
    try:
        urd = _get_url_and_payload(code, None, None, last=2)
        res = requests.get(urd["url"], params=urd["payload"])
        res.raise_for_status()
        data = json_parser.loads(res.text)
        if len(data) < 2: return False
        date_format = "%d/%m/%Y"
        last_date = dt.datetime.strptime(data[1]['data'], date_format).date()
        penultimate_date = dt.datetime.strptime(data[0]['data'], date_format).date()
        return (last_date - penultimate_date).days == 1
    except (requests.RequestException, IndexError, KeyError, ValueError):
        return True

def _fetch_sgs_json(code: int, start: Optional[DateInput], end: Optional[DateInput], last: int) -> str:
    urd = _get_url_and_payload(code, start, end, last)
    res = requests.get(urd["url"], params=urd["payload"])
    res.raise_for_status()
    return res.text

def _fetch_in_chunks(code: int, start_date: dt.date, end_date: dt.date) -> pd.DataFrame:
    df_list = []
    current_start = start_date
    while current_start < end_date:
        current_end = current_start + relativedelta(years=5)
        if current_end > end_date:
            current_end = end_date
        try:
            json_text = _fetch_sgs_json(code, current_start, current_end, 0)
            if json_text:
                chunk_df = pd.read_json(StringIO(json_text), orient='records')
                if not chunk_df.empty:
                    df_list.append(chunk_df)
        except (json_parser.JSONDecodeError, requests.exceptions.HTTPError):
            pass # Ignora blocos com resposta vazia ou erros
        current_start = current_end + relativedelta(days=1)
    if not df_list:
        return pd.DataFrame()
    return pd.concat(df_list, ignore_index=True)

# AJUSTE FINAL E DEFINITIVO ESTÁ NESTA FUNÇÃO
def get_json_as_df(code: int, start: Optional[DateInput] = None, end: Optional[DateInput] = None, last: int = 0) -> pd.DataFrame:
    if last > 0:
        text = _fetch_sgs_json(code, None, None, last)
        return pd.read_json(StringIO(text), orient='records')

    if start:
        start_date = pd.to_datetime(start).date()
        end_date = pd.to_datetime(end or dt.date.today()).date()
        is_long_period = (end_date - start_date).days > 3600
        if is_long_period and _probe_is_daily(code):
            print(f"Série {code}: Período longo e diário detectado. Buscando em blocos...")
            return _fetch_in_chunks(code, start_date, end_date)
        text = _fetch_sgs_json(code, start_date, end_date, 0)
        return pd.read_json(StringIO(text), orient='records')
    else:
        # Tenta o caminho otimista. Se falhar por erro de HTTP ou por erro de parsing do pandas,
        # cai no fallback robusto de busca em blocos.
        try:
            text = _fetch_sgs_json(code, None, None, 0)
            return pd.read_json(StringIO(text), orient='records')
        except (requests.exceptions.HTTPError, ValueError):
            print(f"Série {code}: Requisição inicial falhou ou resultado inválido. Reiniciando busca em blocos...")
            start_date = dt.date(1980, 1, 1)
            end_date = dt.date.today()
            return _fetch_in_chunks(code, start_date, end_date)

# --- 4. FUNÇÃO 'get' FINAL ---

def get(
    codes: SGSCodeInput,
    start: Optional[DateInput] = None,
    end: Optional[DateInput] = None,
    last: int = 0,
    multi: bool = True,
    freq: Optional[str] = None,
) -> Union[pd.DataFrame, List[pd.DataFrame]]:
    dfs = []
    for code in _codes(codes):
        raw_df = get_json_as_df(code.value, start, end, last)
        if raw_df.empty:
            continue
        df = _format_df(raw_df, code, freq)
        if not df.index.is_unique:
            df = df.loc[~df.index.duplicated(keep='first')]
        if start:
            start_date_dt = pd.to_datetime(start)
            df = df.loc[df.index >= start_date_dt]
        dfs.append(df)
    if not dfs:
        return pd.DataFrame()
    if len(dfs) == 1:
        return dfs[0]
    else:
        if multi:
            return pd.concat(dfs, axis=1)
        else:
            return dfs

In [161]:
# --- 1. IMPORTAÇÕES ---
import json as json_parser
from io import StringIO
from typing import Dict, Generator, List, Optional, Tuple, TypeAlias, Union
import datetime as dt

import pandas as pd
import requests
from dateutil.relativedelta import relativedelta

from bcb.utils import Date, DateInput


# --- 2. CLASSES E FUNÇÕES DE BASE ---
class SGSCode:
    """Representa um código de série temporal do SGS com nome e valor."""
    def __init__(self, code: Union[str, int], name: Optional[str] = None) -> None:
        if name is None:
            if isinstance(code, int) or isinstance(code, str):
                self.name = str(code)
                self.value = int(code)
        else:
            self.name = str(name)
            self.value = int(code)

    def __repr__(self):
        return f"{self.value} - {self.name}" if self.name else f"{self.value}"


SGSCodeInput: TypeAlias = Union[int, str, Tuple[str, Union[int, str]], List[Union[int, str, Tuple[str, Union[int, str]]]], Dict[str, Union[int, str]],]


def _codes(codes: SGSCodeInput) -> Generator[SGSCode, None, None]:
    """
    Normaliza diferentes formatos de entrada de códigos de séries em um gerador
    de objetos SGSCode.
    """
    if isinstance(codes, int) or isinstance(codes, str):
        yield SGSCode(codes)
    elif isinstance(codes, tuple):
        yield SGSCode(codes[1], codes[0])
    elif isinstance(codes, list):
        for cd in codes:
            _ist = isinstance(cd, tuple)
            yield SGSCode(cd[1], cd[0]) if _ist else SGSCode(cd)
    elif isinstance(codes, dict):
        for name, code in codes.items():
            yield SGSCode(code, name)


def _get_url_and_payload(code: int, start_date: Optional[DateInput], end_date: Optional[DateInput], last: int) -> Dict[str, str]:
    """Monta a URL e os parâmetros para a chamada à API do SGS."""
    payload = {"formato": "json"}
    if last == 0:
        if start_date is not None or end_date is not None:
            payload["dataInicial"] = Date(start_date).date.strftime("%d/%m/%Y")
            end_date = end_date if end_date else "today"
            payload["dataFinal"] = Date(end_date).date.strftime("%d/%m/%Y")
        url = f"https://api.bcb.gov.br/dados/serie/bcdata.sgs.{code}/dados"
    else:
        url = f"https://api.bcb.gov.br/dados/serie/bcdata.sgs.{code}/dados/ultimos/{last}"
    return {"payload": payload, "url": url}


def _format_df(df: pd.DataFrame, code: SGSCode, freq: str) -> pd.DataFrame:
    """
    Formata o DataFrame bruto retornado pela API, renomeando colunas,
    convertendo tipos de dados e definindo o índice.
    """
    cns = {"data": "Date", "valor": code.name, "datafim": "enddate"}
    df = df.rename(columns=cns)
    if "Date" in df:
        df["Date"] = pd.to_datetime(df["Date"], dayfirst=True)
    if "enddate" in df:
        df["enddate"] = pd.to_datetime(df["enddate"], dayfirst=True)
    df = df.set_index("Date")
    if freq:
        df.index = df.index.to_period(freq)
    return df


# --- 3. FUNÇÕES AUXILIARES PARA A LÓGICA DE BUSCA AVANÇADA ---

def _probe_is_daily(code: int) -> bool:
    """
    Sonda a API buscando os 2 últimos pontos para determinar se a série é diária.
    A verificação é feita calculando a diferença em dias entre os dois últimos pontos.

    Args:
        code (int): O código da série a ser verificada.

    Returns:
        bool: True se a diferença for de 1 dia (indicando série diária),
              False caso contrário. Em caso de erro, assume True por segurança.
    """
    try:
        # Pede os dois últimos pontos da série para inferir a frequência.
        urd = _get_url_and_payload(code, None, None, last=2)
        res = requests.get(urd["url"], params=urd["payload"])
        res.raise_for_status()
        data = json_parser.loads(res.text)

        # Se houver menos de 2 pontos, não é possível inferir a frequência.
        if len(data) < 2:
            return False

        # Converte as datas (formato "DD/MM/YYYY") e calcula a diferença.
        date_format = "%d/%m/%Y"
        last_date = dt.datetime.strptime(data[1]['data'], date_format).date()
        penultimate_date = dt.datetime.strptime(data[0]['data'], date_format).date()
        delta_days = (last_date - penultimate_date).days
        return delta_days <= 20
    except (requests.RequestException, IndexError, KeyError, ValueError):
        # Se a sondagem falhar, assume o pior cenário (série diária) para
        # garantir que a busca em blocos seja acionada se o período for longo.
        return True


def _fetch_sgs_json(code: int, start: Optional[DateInput], end: Optional[DateInput], last: int) -> str:
    """
    Executa uma única requisição HTTP para a API do SGS e retorna o JSON como texto.

    Args:
        code (int): Código da série.
        start (DateInput, optional): Data de início.
        end (DateInput, optional): Data de fim.
        last (int): Número de últimas observações a serem retornadas.

    Returns:
        str: A resposta da API em formato JSON de texto.
    """
    urd = _get_url_and_payload(code, start, end, last)
    res = requests.get(urd["url"], params=urd["payload"])
    res.raise_for_status()  # Lança um erro para status HTTP de falha (4xx ou 5xx)
    return res.text


def _fetch_in_chunks(code: int, start_date: dt.date, end_date: dt.date) -> pd.DataFrame:
    """
    Busca os dados de uma série em blocos de tempo para contornar limites da API.
    Converte cada bloco em um DataFrame e os concatena no final.

    Args:
        code (int): Código da série.
        start_date (dt.date): Data de início do período total.
        end_date (dt.date): Data de fim do período total.

    Returns:
        pd.DataFrame: Um DataFrame contendo todos os dados do período solicitado.
    """
    df_list = []
    current_start = start_date
    while current_start < end_date:
        # Define o fim do bloco (chunk) atual.
        current_end = current_start + relativedelta(years=5)
        if current_end > end_date:
            current_end = end_date
        
        try:
            json_text = _fetch_sgs_json(code, current_start, current_end, 0)
            if json_text:
                # Converte o JSON do bloco em um DataFrame e o adiciona à lista.
                chunk_df = pd.read_json(StringIO(json_text), orient='records')
                if not chunk_df.empty:
                    df_list.append(chunk_df)
        except (json_parser.JSONDecodeError, requests.exceptions.HTTPError):
            # Ignora blocos que retornam resposta vazia ou com erro, continuando o processo.
            pass
            
        # Avança para o próximo bloco.
        current_start = current_end + relativedelta(days=1)
    
    if not df_list:
        return pd.DataFrame()
    
    return pd.concat(df_list, ignore_index=True)


def get_json_as_df(code: int, start: Optional[DateInput] = None, end: Optional[DateInput] = None, last: int = 0) -> pd.DataFrame:
    """
    Função central que orquestra a busca de dados, decidindo entre uma busca
    única ou em blocos, e sempre retorna um DataFrame.

    Args:
        code (int): Código da série.
        start (DateInput, optional): Data de início.
        end (DateInput, optional): Data de fim.
        last (int): Número de últimas observações.

    Returns:
        pd.DataFrame: DataFrame com os dados brutos da série ("data", "valor").
    """
    # Caso 1: Busca pelas últimas 'n' observações.
    if last > 0:
        text = _fetch_sgs_json(code, None, None, last)
        return pd.read_json(StringIO(text), orient='records')

    # Caso 2: Datas de início e fim são fornecidas.
    if start:
        start_date = pd.to_datetime(start).date()
        end_date = pd.to_datetime(end or dt.date.today()).date()
        is_long_period = (end_date - start_date).days > 3600
        
        # Lógica proativa: se o período for longo, sonda a frequência.
        if is_long_period and _probe_is_daily(code):
            return _fetch_in_chunks(code, start_date, end_date)
        
        # Se não for longo ou não for diário, faz uma busca única.
        text = _fetch_sgs_json(code, start_date, end_date, 0)
        return pd.read_json(StringIO(text), orient='records')
    
    # Caso 3: Nenhuma data fornecida (busca a série completa).
    else:
        # Lógica reativa: tenta a busca completa. Se falhar, assume que é uma
        # série diária longa e recorre à busca em blocos.
        try:
            text = _fetch_sgs_json(code, None, None, 0)
            return pd.read_json(StringIO(text), orient='records')
        except (requests.exceptions.HTTPError, ValueError):
            # O fallback inicia a busca a partir de uma data antiga e segura.
            start_date = dt.date(1980, 1, 1)
            end_date = dt.date.today()
            return _fetch_in_chunks(code, start_date, end_date)


# --- 4. FUNÇÃO PRINCIPAL (INTERFACE PÚBLICA) ---

def get(
    codes: SGSCodeInput,
    start: Optional[DateInput] = None,
    end: Optional[DateInput] = None,
    last: int = 0,
    multi: bool = True,
    freq: Optional[str] = None,
) -> Union[pd.DataFrame, List[pd.DataFrame]]:
    """
    Retorna um DataFrame pandas com séries temporais obtidas do SGS.
    Esta função contorna a limitação de 10 anos para séries diárias ao
    realizar múltiplas requisições em blocos quando necessário.

    Args:
        codes ({int, str, list, dict}): Código(s) da(s) série(s) a ser(em) consultada(s).
        start (DateInput, optional): Data de início da série.
        end (DateInput, optional): Data de fim da série.
        last (int, optional): Retorna os 'n' últimos dados disponíveis.
        multi (bool, optional): Se True, retorna um único DataFrame para múltiplas
                                séries. Se False, uma lista de DataFrames.
        freq (str, optional): Frequência a ser utilizada no índice do DataFrame.

    Returns:
        Union[pd.DataFrame, List[pd.DataFrame]]: DataFrame (ou lista de DataFrames)
                                                 com as séries temporais.
    """
    dfs = []
    for code in _codes(codes):
        # A função get_json_as_df abstrai toda a complexidade da busca.
        raw_df = get_json_as_df(code.value, start, end, last)
        
        if raw_df.empty:
            continue
        
        # Formata o DataFrame bruto para o padrão final.
        df = _format_df(raw_df, code, freq)
        
        # Garante a remoção de duplicatas no índice (uma camada extra de segurança).
        if not df.index.is_unique:
            df = df.loc[~df.index.duplicated(keep='first')]
        
        # Garante que os dados retornados comecem na data de início solicitada.
        if start:
            start_date_dt = pd.to_datetime(start)
            df = df.loc[df.index >= start_date_dt]
        
        dfs.append(df)
        
    if not dfs:
        return pd.DataFrame()
    
    if len(dfs) == 1:
        return dfs[0]
    else:
        if multi:
            return pd.concat(dfs, axis=1)
        else:
            return dfs

In [ ]:
len(get_json(433, start='2009-03-17', end='2025-07-17'))

2858

In [89]:
json = get_json(433, start='2019-03-17', end='2025-07-17') \
    #.replace('[', '').replace(']', '')#.replace('{', '').replace('}', '') \
        # .split(',')

json
# for i in json: 
#     print(i)
#     print('')

'[{"data":"01/03/2019","valor":"0.75"},{"data":"01/04/2019","valor":"0.57"},{"data":"01/05/2019","valor":"0.13"},{"data":"01/06/2019","valor":"0.01"},{"data":"01/07/2019","valor":"0.19"},{"data":"01/08/2019","valor":"0.11"},{"data":"01/09/2019","valor":"-0.04"},{"data":"01/10/2019","valor":"0.10"},{"data":"01/11/2019","valor":"0.51"},{"data":"01/12/2019","valor":"1.15"},{"data":"01/01/2020","valor":"0.21"},{"data":"01/02/2020","valor":"0.25"},{"data":"01/03/2020","valor":"0.07"},{"data":"01/04/2020","valor":"-0.31"},{"data":"01/05/2020","valor":"-0.38"},{"data":"01/06/2020","valor":"0.26"},{"data":"01/07/2020","valor":"0.36"},{"data":"01/08/2020","valor":"0.24"},{"data":"01/09/2020","valor":"0.64"},{"data":"01/10/2020","valor":"0.86"},{"data":"01/11/2020","valor":"0.89"},{"data":"01/12/2020","valor":"1.35"},{"data":"01/01/2021","valor":"0.25"},{"data":"01/02/2021","valor":"0.86"},{"data":"01/03/2021","valor":"0.93"},{"data":"01/04/2021","valor":"0.31"},{"data":"01/05/2021","valor":"0.8

In [164]:
# from bcb import sgs
import pandas as pd
import numpy as np
series = dict(
            # cambio=1, 
            # selic=11,
            ipca=433
            )

df_bcb  = get(series, start='2000-03-17', end='2025-08-17')
(
    df_bcb
    # .plot(subplots=True, figsize=(10,4))
)

,ipca
Date,
2000-04-01,0.42
2000-05-01,0.01
2000-06-01,0.23
2000-07-01,1.61
2000-08-01,1.31
...,...
2025-03-01,0.56
2025-04-01,0.43
2025-05-01,0.26


In [122]:
df_bcb#.reset_index().value_counts()

In [51]:
from bcb import sgs

sgs.get({'ipca':1}, last=5)

,ipca
Date,
2025-08-11,5.4473
2025-08-12,5.4052
2025-08-13,5.3928
2025-08-14,5.4095
2025-08-15,5.3928


não faz sentido identificar se a série é diária ou não pelo número de '/'



```

from bcb import sgs

sgs.get({'ipca':433}, last=1)

```

Retorna 

Data | ipca

2025-07-01 | 0.26

 e é uma série mensal



assim como



```

from bcb import sgs

sgs.get({'cambio':1}, last=1)

```

retorna 

Data | ipca

2025-08-15 | 5.3928



e é diária

In [32]:
get({'ipca':433}, start = '2000-01-01')

Start: 1999-01-01 | End: 2004-01-01
Start: 2004-01-01 | End: 2009-01-01
Start: 2009-01-01 | End: 2014-01-01
Start: 2014-01-01 | End: 2019-01-01
Start: 2019-01-01 | End: 2024-01-01
Start: 2024-01-01 | End: 2025-08-17


,ipca
Date,
2000-01-01,0.62
2000-02-01,0.13
2000-03-01,0.22
2000-04-01,0.42
2000-05-01,0.01
...,...
2025-03-01,0.56
2025-04-01,0.43
2025-05-01,0.26


In [16]:
sgs.get({'ipca':433}, start='1899-01-01', end='1904-01-01' )

Exception: BCB error: br.gov.bcb.pec.sgs.comum.excecoes.SGSNegocioException: Value(s) not found

In [220]:
cambio.drop_duplicates(ignore_index=False)

,cambio,selic
Date,,
2000-01-03,1.8011,0.069186
2000-01-04,1.8337,NaN
2000-01-05,1.8544,0.069220
2000-01-06,1.8461,0.069286
2000-01-07,1.8281,NaN
...,...,...
2025-08-07,5.4638,NaN
2025-08-11,5.4473,NaN
2025-08-12,5.4052,NaN


In [5]:
start = '2000-03-17'

end = dt.date.today()


start_as_date = dt.datetime.strptime(start, '%Y-%m-%d').date() if type(start) == str else start

end_as_date = dt.datetime.strptime(end, '%Y-%m-%d').date() if type(end) == str else end

print("Start:", start_as_date, "\nEnd:", end_as_date)

Start: 2000-03-17 
End: 2025-08-17


In [101]:
diff_years = (end_as_date - start_as_date).days / 365

diff_years > 8

True

In [ ]:
?pd.date_range

In [ ]:
?dt.timedelta

In [102]:
(start_as_date - pd.offsets.YearBegin(1)).date()

datetime.date(2000, 1, 1)

In [35]:
previous_start_year = (start_as_date - pd.offsets.YearBegin(1)).date()
next_end_year = (end_as_date + pd.offsets.YearBegin(0)).date()

In [36]:
previous_start_year

datetime.date(2000, 1, 1)

In [42]:
intervals = pd.date_range(start=previous_start_year, end=next_end_year, freq='8YS', inclusive='left').to_list()
intervals = intervals + [end]
intervals

[Timestamp('2000-01-01 00:00:00'),
 Timestamp('2008-01-01 00:00:00'),
 Timestamp('2016-01-01 00:00:00'),
 Timestamp('2024-01-01 00:00:00'),
 '2025-07-17']

In [46]:
series = dict(cambio=1)
final_df = pd.DataFrame()

for i_dates in range(len(intervals)-1):
    print(i_dates)

    first_date = intervals[i_dates]
    print(first_date)

    last_date = intervals[i_dates+1]
    print(last_date)

    query = get(series, start=first_date, end=last_date)

    final_df = pd.concat([final_df, query], axis=0)

final_df.drop_duplicates(inplace=True)
final_df = final_df.loc[start:]
final_df



0
2000-01-01 00:00:00
2008-01-01 00:00:00
1
2008-01-01 00:00:00
2016-01-01 00:00:00
2
2016-01-01 00:00:00
2024-01-01 00:00:00
3
2024-01-01 00:00:00
2025-07-17


TypeError: unsupported operand type(s) for -: 'datetime.date' and 'Timestamp'

In [168]:
pd.date_range(start=start, end=end, freq='8YS')

DatetimeIndex(['2001-01-01', '2009-01-01', '2017-01-01', '2025-01-01'], dtype='datetime64[ns]', freq='8YS-JAN')

In [156]:
years_insample = 8
n_periods = int((end_as_date - start_as_date).days/365)
n_periods

25

In [167]:
diff_years/ (diff_years/years_insample)

8.0

In [169]:
diff_years / years_insample

3.1794520547945204

In [173]:
(end_as_date - start_as_date).days/8

1160.5

In [ ]:
periods/years = freq

In [174]:
25*8

200